# The simplest model

How, then, does a disease become an epidemic? More precisely, what are the characteristics of a disease that control how many people in a population become infected?

Clearly this is a complicated question, and a lot of factors will affect the answer. Even very subtle things might be important, and may interact with features of the disease itself: when people meet, for example, do they kiss, or do they shake hands? The former will make it easier to pass a disease that needs close physical contact in order to spread, but might make less difference to an airborne disease, a no difference at all to a disease spread by mosquitoes.

A scientist's usual approach when faced with a situation like this is try to find a *model* to analyse. By a model we simply mnean a mathematical and/or computational 

In [116]:
import numpy
import operator

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn

from ipywidgets import interact, fixed
import ipywidgets as widgets

$$
    \Delta S = -\beta SI \qquad \qquad
    \Delta I = \beta SI - \alpha I \qquad \qquad
    \Delta R = \alpha I =
$$

In [117]:
def make_sir(beta, alpha):
    # turn the equations into update functions
    def dS(S, I, R):
        return -beta * S * I
    def dI(S, I, R):
        return beta * S * I - alpha * I
    def dR(S, I, R):
        return alpha * I
    
    # return the three functions
    return (dS, dI, dR)

In [123]:
def epidemic_sir(T, N, pInfected, pInfect, pRemove):
    # create the equations for these parameters
    (dS, dI, dR) = make_sir(pInfect, pRemove)
    
    # initial conditions
    sss = [ N * (1.0 - pInfected) ]   # everyone is initially susceptible...
    iss = [ N * pInfected ]           # ...except for this fraction...
    rss = [ 0 ]                       # ...and no-one starts off removed

    # push the initial conditions through the differential equations
    for t in range(1, T):
        ds = dS(sss[-1], iss[-1], rss[-1])
        di = dI(sss[-1], iss[-1], rss[-1])
        dr = dR(sss[-1], iss[-1], rss[-1])
        sss.append(sss[-1] + ds)
        iss.append(iss[-1] + di)
        rss.append(rss[-1] + dr)
        
    # return the time series
    return (list(range(0, T)), sss, iss, rss)

In [119]:
N = 1000
T = 5000
pInfected = 0.01

In [124]:
@interact(pInfected=fixed(pInfected), \
          pInfect=widgets.FloatSlider(min=0.0, max=0.01, step=(0.01 / 20), value=0.004, \
                                      description='$\\beta \\times 1000$', continuous_update=False, readout_format='.4f'), \
          pRemove=widgets.FloatSlider(min=0.0, max=0.004, step=(0.004 / 20), value=0.001, \
                                      description='$\\alpha$', continuous_update=False, readout_format='.4f'))
def interact_sir(pInfected, pInfect, pRemove):
    # run the epidemic equations
    (ts, sss, iss, rss) = epidemic_sir(T, N, pInfected, pInfect / 1000, pRemove)
    
    # draw the graph
    fig = plt.figure(figsize=(8, 6))
    plt.plot(ts, sss, 'r-', label='suceptible')
    plt.plot(ts, iss, 'g-', label='infected')
    plt.title('Progress of epidemic ($\\beta = {b}, \\alpha = {a}$)'.format(b=pInfect, a=pRemove))
    plt.xlabel('$t$')
    plt.xlim([0, T])
    plt.ylabel('population that is...')
    plt.ylim([0, N])
    plt.legend(loc='right')
    plt.show()

interactive(children=(FloatSlider(value=0.004, continuous_update=False, description='$\\beta \\times 1000$', m…

\begin{align}
    0 &= \beta SI - \alpha I \\
      &= I(\beta S - \alpha)
\end{align}

Since $I$ cannot be negative, then either $I = 0$ (there are no infected individuals left in the population) or $\beta S = \alpha$ meaning $S = \frac{\alpha}{\beta}$.

In [121]:
def closestTo(ss, target):
    ds = None
    sc = None
    ic = None
    for i in range(len(ss)):
        df = abs(ss[i] - target)
        if ds is None or df < ds:
            ds = df
            sc = ss[i]
            ic = i
    return (ic, sc)

In [122]:
@interact(pInfected=fixed(0.01), \
          pInfect=widgets.FloatSlider(min=0.0, max=0.01, step=(0.01 / 20), value=0.004, \
                                      description='$\\beta \\times 1000$', continuous_update=False, readout_format='.4f'), \
          pRemove=widgets.FloatSlider(min=0.0, max=0.004, step=(0.004 / 20), value=0.001, \
                                      description='$\\alpha$', continuous_update=False, readout_format='.4f'))
def interact_sir(pInfected, pInfect, pRemove):
    # run the epidemic equations
    (ts, sss, iss, rss) = epidemic_sir(T, N, pInfected, pInfect / 1000, pRemove)
    
    # compute the critical S
    scrit = pRemove / (pInfect / 1000)
    (tcrit, _) = closestTo(sss, scrit)

    # draw the graph
    fig = plt.figure(figsize=(8, 6))
    plt.plot(ts, sss, 'r-', label='suceptible')
    plt.plot(ts, iss, 'g-', label='infected')
    plt.plot([0, T], [scrit, scrit], 'k--')
    plt.plot([tcrit, tcrit], [0.0, N], 'k--')    
    plt.title('Progress of epidemic ($\\beta = {b}, \\alpha = {a}$)'.format(b=pInfect, a=pRemove))
    plt.xlabel('$t$')
    plt.xlim([0, T])
    plt.ylabel('population that is...')
    plt.ylim([0.0, N])
    plt.legend(loc='right')
    plt.show()

interactive(children=(FloatSlider(value=0.004, continuous_update=False, description='$\\beta \\times 1000$', m…